# HackRF

https://hackrf.readthedocs.io/en/latest/hackrf_tools.html

In [1]:
!hackrf_info

hackrf_info version: 2024.02.1
libhackrf version: 2024.02.1 (0.9)
Found HackRF
Index: 0
Serial number: 0000000000000000f77c60dc292134c3
Board ID Number: 2 (HackRF One)
Firmware Version: 2024.02.1 (API:1.08)
Part ID Number: 0xa000cb3c 0x00614758
Hardware Revision: older than r6
Hardware supported by installed firmware:
    HackRF One


In [30]:
!hackrf_sweep -f 88:106 -N 10 -w 100000 -r ./data/n10f88-106.csv

call hackrf_sample_rate_set(20.000 MHz)
call hackrf_baseband_filter_bandwidth_set(15.000 MHz)
Sweeping from 88 MHz to 108 MHz
Stop with Ctrl-C

Exiting...
Total sweeps: 10 in 0.05010 seconds (199.62 sweeps/second)
hackrf_close() done
hackrf_exit() done
fclose() done
exit


In [10]:
!hackrf_transfer

specify one of: -t, -c, -r, -w
Usage:
	-h # this help
	[-d serial_number] # Serial number of desired HackRF.
	-r <filename> # Receive data into file (use '-' for stdout).
	-t <filename> # Transmit data from file (use '-' for stdin).
	-w # Receive data into file with WAV header and automatic name.
	   # This is for SDR# compatibility and may not work with other software.
	[-f freq_hz] # Frequency in Hz [1MHz to 6000MHz supported, 0MHz to 7250MHz forceable].
	[-i if_freq_hz] # Intermediate Frequency (IF) in Hz [2170MHz to 2740MHz supported, 2000MHz to 3000MHz forceable].
	[-o lo_freq_hz] # Front-end Local Oscillator (LO) frequency in Hz [84MHz to 5400MHz].
	[-m image_reject] # Image rejection filter selection, 0=bypass, 1=low pass, 2=high pass.
	[-a amp_enable] # RX/TX RF amplifier 1=Enable, 0=Disable.
	[-p antenna_enable] # Antenna port power, 1=Enable, 0=Disable.
	[-l gain_db] # RX LNA (IF) gain, 0-40dB, 8dB steps
	[-g gain_db] # RX VGA (baseband) gain, 0-62dB, 2dB steps
	[-x gain_db] 

In [19]:
!hackrf_transfer -f 91100000 -w -n 100

Receive wav file: HackRF_20240812_222438Z_91100kHz_IQ.wav
call hackrf_set_sample_rate(10000000 Hz/10.000 MHz)
call hackrf_set_hw_sync_mode(0)
call hackrf_set_freq(91100000 Hz/91.100 MHz)
samples_to_xfer 100/0Mio
Stop with Ctrl-C
 0.3 MiB / 0.014 sec = 19.2 MiB/second, average power -34.4 dBfs

Exiting...
Total time: 0.01366 s
hackrf_stop_rx() done
hackrf_close() done
hackrf_exit() done
fclose() done
exit
